# Intro agli LLM

https://animatedllm.github.io/

## Interazione con gli LLM tramite LangChain

In [ ]:
!pip install langchain langchain-core langchain-openai openai pandas langchain-community pypdf fastembed langchain-qdrant qdrant-client

In [10]:
my_api_key = 'xxxx'

In [11]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import SystemMessage, HumanMessage

llm = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    max_tokens=1000,
    api_key = my_api_key
)


messages = [
    ("system","Sei un assistente tecnico e conciso."),
    ("human","Cos'è LangChain?")
]


response = llm.invoke(input = messages)
print(response.content)

LangChain è un framework progettato per facilitare lo sviluppo di applicazioni basate su modelli di linguaggio, come quelli di OpenAI. Consente di integrare vari componenti come modelli, sorgenti di dati, strumenti e interfacce utente, permettendo di costruire applicazioni modulari e scalabili. È particolarmente utile per creare chatbot, assistenti virtuali e altre applicazioni che richiedono interazioni linguistiche avanzate e gestione del contesto.


Osserviamo il tipo dell'oggetto response

In [5]:
type(response)

langchain_core.messages.ai.AIMessage

Esempio banale

In [6]:
#gli sto passando una lista con solo uno HumanMessage
#llm.invoke("come stai?").content

llm.invoke(input=[("human","come stai?")]).content

'Sto bene, grazie! E tu come stai?'

## Dinamizziamo il prompt

In [13]:
from langchain_core.prompts import ChatPromptTemplate

llm = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    api_key = my_api_key,
    max_tokens=1000
)

prompt = ChatPromptTemplate.from_messages(messages = [
    ("system", "Sei un assistente tecnico e molto conciso."),
    ("human", "Spiega {argomento} in una frase.")
])

messages = prompt.invoke(input={"argomento": "LangChain"})
response = llm.invoke(input = messages)

print(response.content)

LangChain è una libreria per sviluppare applicazioni basate su modelli linguistici, facilitando l'integrazione di vari strumenti e fonti di dati.


In [14]:
type(messages)

langchain_core.prompt_values.ChatPromptValue

Facciamo meglio con una catena!

In [16]:
from langchain_core.prompts import ChatPromptTemplate

llm = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    api_key = my_api_key,
    max_tokens=1000
)

prompt = ChatPromptTemplate.from_messages(messages = [
    ("system", "Sei un assistente tecnico e molto conciso."),
    ("human", "Spiega {argomento} in una frase.")
])

chain = prompt | llm

response = chain.invoke(input={"argomento": "Python"})
print(response.content)

Python è un linguaggio di programmazione ad alto livello, versatile e di facile lettura, utilizzato per sviluppo web, analisi dati, intelligenza artificiale e automazione.


Esempio con secondo parametro

In [19]:
from langchain_core.prompts import ChatPromptTemplate

llm = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    api_key = my_api_key,
    max_tokens=1000
    )

prompt = ChatPromptTemplate.from_messages(messages = [
    ("system", "Sei un assistente tecnico e molto conciso."),
    ("human", "Spiega {argomento} in una frase. Spiegalo come se il tuo interlocutore ha {anni} anni")
])

chain = prompt | llm

response = chain.invoke(input={"argomento": "Python","anni":"8"})
print(response.content)

Python è un linguaggio di programmazione semplice e divertente che permette di dire al computer cosa fare, un po' come scrivere istruzioni per un gioco!


## Inseriamo un parser alla fine!

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    api_key = my_api_key,
    max_tokens=1000
)

prompt = ChatPromptTemplate.from_messages(messages = [
    ("system", "Sei un assistente tecnico e molto conciso."),
    ("human", "Spiega {argomento} in una frase.")
])

chain = prompt | llm | StrOutputParser()

response = chain.invoke({"argomento": "LangChain"})

#response ora è già un oggetto di tipo stringa!
response

"LangChain è una libreria per la creazione di applicazioni basate su modelli di linguaggio, facilitando l'integrazione con vari strumenti e fonti di dati."

Proviamo ad avere un output json...otteremo un errore

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

llm = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    api_key = my_api_key,
    max_tokens=1000
)

prompt = ChatPromptTemplate.from_messages(messages = [
    ("system", "Sei un assistente tecnico e molto conciso."),
    ("human", "Spiega {argomento} in una frase.")
])

chain = prompt | llm | JsonOutputParser()

response = chain.invoke({"argomento": "LangChain"})

#response ora è già un oggetto di tipo dict!
response

OutputParserException: Invalid json output: LangChain è una libreria progettata per semplificare la creazione di applicazioni basate su modelli di linguaggio, integrando strumenti di NLP e fonti di dati.
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 

Proviamo un altro esempio con un prompt diverso

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    api_key = my_api_key,
    max_tokens=1000
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Rispondi SOLO con JSON valido, senza testo extra."),
    ("human", "Crea un oggetto JSON con chiavi: titolo, punti (lista di stringhe), difficolta.\nTema: {tema}")
])

chain = prompt | llm | JsonOutputParser()

response = chain.invoke({"tema": "LangChain"})
print(response)

{'titolo': 'Introduzione a LangChain', 'punti': ["Cos'è LangChain e a cosa serve", 'Architettura di LangChain', 'Installazione di LangChain', 'Esempi di utilizzo di LangChain', 'Integrazione con modelli di linguaggio'], 'difficolta': 'Media'}


Controlliamo il tipo di response!

In [24]:
type(response)

dict

## Esempio con CSV

In [25]:
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableLambda
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    api_key = my_api_key,
    max_tokens=1000
)

parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_messages([
    ("system", "Rispondi SOLO con JSON valido, senza testo extra."),
    ("human",
     "Genera una lista spesa per: {tema}. "
     "Rispondi con un JSON che abbia ESATTAMENTE questa forma: "
     '{{"righe":[{{"nome":"string","quantita":1,"prezzo_eur":1.23}}]}} '
     "Non aggiungere altro testo.")
])

def json_to_csv(data):
    df = pd.DataFrame(data["righe"])
    return df.to_csv(index=False, sep = ";")

csv_converter = RunnableLambda(json_to_csv)

chain = prompt | llm | parser | csv_converter

csv_text = chain.invoke({"tema": "colazione italiana"})
print(csv_text)

nome;quantita;prezzo_eur
caffè;1;0.5
latte;1;0.8
cornetto;2;1.2
zucchero;1;0.1
biscotti;1;1.0



## Memoria sequenziale

In [26]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

llm = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    api_key = my_api_key,
    max_tokens=1000
)

messages = [
    SystemMessage(content="Sei un assistente tecnico e conciso."),

    HumanMessage(content="Cos'è LangChain?"),
    AIMessage(content="LangChain è un framework Python per costruire applicazioni basate su LLM."),

    HumanMessage(content="A cosa serve in pratica?")
]

messages = [
    ("system","Sei un assistente tecnico e conciso."),

    ("human", "Cos'è LangChain?"),
    ("ai", "LangChain è un framework Python per costruire applicazioni basate su LLM."),

    ("human", "A cosa serve in pratica?"),
]

response = llm.invoke(input=messages)
print(response.content)

In pratica, LangChain serve a creare applicazioni che utilizzano modelli di linguaggio per elaborare il testo, generare contenuti, rispondere a domande, gestire conversazioni e integrarli con altre fonti di dati o API. Facilita la costruzione di flussi di lavoro complessi che coinvolgono il trattamento del linguaggio naturale.


Vediamo come si gestisce con langchain

In [27]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 1) LLM
llm = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    max_tokens=1000,
    api_key = my_api_key
)

# 2) Prompt: include la history in mezzo
prompt = ChatPromptTemplate.from_messages([
    ("system", "Sei un assistente tecnico e conciso."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

chain = prompt | llm

# 3) Store delle sessioni
_store = {}

def get_history(session_id):
    if session_id not in _store:
        _store[session_id] = InMemoryChatMessageHistory()
    return _store[session_id]

# 4) chain_with_memory
chain_with_memory = RunnableWithMessageHistory(
    runnable = chain,
    get_session_history = get_history,
    input_messages_key="input",
    history_messages_key="history",
)

# 5) Usa una sessione (es. "nicola_202601211418")
config = {"configurable": {"session_id": "nicola_202601211418"}}

print(chain_with_memory.invoke({"input": "Ciao, mi chiamo Nicola."}, config=config).content)
print(chain_with_memory.invoke({"input": "Come mi chiamo?"}, config=config).content)
print(chain_with_memory.invoke({"input": "Riassumi cosa abbiamo detto finora in 1 frase."}, config=config).content)

Ciao Nicola! Come posso aiutarti oggi?
Ti chiami Nicola.
Hai detto di chiamarti Nicola e io ti ho confermato il tuo nome.


Ulteriore test

In [28]:
print(chain_with_memory.invoke({"input": "Come mi chiamo?"}, config=config).content)

Ti chiami Nicola.


Visualizziamo la history di nicola_202601211418

In [29]:
history_nicola = _store["nicola_202601211418"]

In [30]:
type(history_nicola)

langchain_core.chat_history.InMemoryChatMessageHistory

In [31]:
for msg in history_nicola.messages:
    print(type(msg).__name__, "->", msg.content)

HumanMessage -> Ciao, mi chiamo Nicola.
AIMessage -> Ciao Nicola! Come posso aiutarti oggi?
HumanMessage -> Come mi chiamo?
AIMessage -> Ti chiami Nicola.
HumanMessage -> Riassumi cosa abbiamo detto finora in 1 frase.
AIMessage -> Hai detto di chiamarti Nicola e io ti ho confermato il tuo nome.
HumanMessage -> Come mi chiamo?
AIMessage -> Ti chiami Nicola.


Proviamo una chiamata con un'altra sessione

In [32]:
config = {"configurable": {"session_id": "giovanni_202601211418"}}

print(chain_with_memory.invoke({"input": "Come mi chiamo?"}, config=config).content)

Non ho accesso alle informazioni personali, quindi non posso sapere come ti chiami. Come posso aiutarti?


In [33]:
for msg in _store["giovanni_202601211418"].messages:
    print(type(msg).__name__, "->", msg.content)

HumanMessage -> Come mi chiamo?
AIMessage -> Non ho accesso alle informazioni personali, quindi non posso sapere come ti chiami. Come posso aiutarti?


In [34]:
config = {"configurable": {"session_id": "giovanni_202601211418"}}

print(chain_with_memory.invoke({"input": "Cosa ti ho chiesto prima?"}, config=config).content)

Mi hai chiesto come ti chiami.


# Database vettoriali

Acquisiamo i pdf

In [35]:
from langchain_community.document_loaders import PyPDFLoader

documents = []

# ======================
# PDF SQL
# ======================
sql_path = r"Database_e_SQL.pdf"
sql_loader = PyPDFLoader(sql_path)
sql_docs = sql_loader.load()

Facciamo qualche analisi sul risultato di sql_loader.load()

In [36]:
type(sql_docs)

list

In [37]:
type(sql_docs[0])

langchain_core.documents.base.Document

In [41]:
print(sql_docs[5].page_content)

Database non 
relazionali
I database non relazionali
sono database in cui i dati non
sono organizzati in tabelle (grafi,
documenti JSON, eccetera)


Popoliamo la lista documents con le pagine dei due pdf

In [42]:
from langchain_community.document_loaders import PyPDFLoader

documents = []

# ======================
# PDF SQL
# ======================
sql_path = r"Database_e_SQL.pdf"
sql_loader = PyPDFLoader(sql_path)
sql_docs = sql_loader.load()

for d in sql_docs:
    d.metadata["corso"] = "sql"
    d.metadata["source"] = sql_path

documents.extend(sql_docs)

# ======================
# PDF PYTHON
# ======================
python_path = r"Python-e-Machine-Learning.pdf"
python_loader = PyPDFLoader(python_path)
python_docs = python_loader.load()

for d in python_docs:
    d.metadata["corso"] = "python"
    d.metadata["source"] = python_path

documents.extend(python_docs)

print(f"Numero pagine caricate: {len(documents)}")

Numero pagine caricate: 305


Dividiamo le pagine in chunk

In [43]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150
)

chunks = splitter.split_documents(documents)
print(f"Chunk totali: {len(chunks)}")

Chunk totali: 428


Qualche analisi sui chang

In [44]:
type(chunks[1])

langchain_core.documents.base.Document

Scarichiamo il modello di embedding

In [45]:
from langchain_community.embeddings import FastEmbedEmbeddings

embeddings = FastEmbedEmbeddings(
    model_name="BAAI/bge-small-en-v1.5"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

Creiamo il DB vettoriale

In [46]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

client = QdrantClient(path=r"qdrant_db")

Creiamo la collezione

In [47]:
client.create_collection(
    collection_name="appunti_corso",
    vectors_config=VectorParams(
        size=384,
        distance=Distance.COSINE
    )
)

True

Carichiamo i vettori creati precedentemente con l'embedding

In [48]:
from langchain_qdrant import QdrantVectorStore

#oggetto di langchain per collegarsi e usare il database
vectorstore = QdrantVectorStore(
    client=client,
    collection_name="appunti_corso",
    embedding=embeddings
)


In [49]:
#aggiunta dei vettori al DB
vectorstore.add_documents(chunks)

['6562c577c8374979a0b8151dc78cbaf6',
 'f4058bd2b0a646d6b59dd8f84c9036cd',
 'fc2c4c7192fd47eab1c4dfdee0be9158',
 '8fdb5d8616e64cacb8978b06f1762f42',
 '992a1ed45696400b9d3f3c25fe49b20c',
 'f515f98650e349cd8185a7b055bd2a4c',
 '7b9f9ea6b3e04b6aad43d120d64cde2e',
 '5a8bc2bfff7e4274ad98eee5844b0d36',
 '8dade368f2f04070ab79a91dd110fb9e',
 '8275f34780ab40409d0ca46b0732f123',
 '32a6a4f1a4914764a1c78c2357158714',
 'af16872979ee4740988066f09caa9744',
 'c1c64b30ba9e413a913f55da4c4d969e',
 '1df44866ca6e4f229e5d25cba6dc43e8',
 '7f2f1372aa7a4863ac252eaf2e9d6f4c',
 '8d257adf78044c5986a94702c95cc741',
 'c7620f1103014f8c9e0e349473400ccd',
 'b62215c2b3ba4b4497e31282d93906dd',
 '9af21e691b1a46368313007946304b9e',
 'c4f16eab039241e7b992166922c6af32',
 '7a00b02488ea400bb5837d33b3d3d315',
 '489e408b6052481b8945b44ee28f1615',
 '9858d9f1967649e49e05efd14bbfa1ea',
 '8dc62e4d59e94546afb16b38ed35ae78',
 'e0205a0225ed43d59bae2828126d9e16',
 '36f66b683ca64657961cc492f44a4693',
 '6da612486f914edaa594cb7834ab7bac',
 

Facciamo una sorta di count(*)

In [50]:
count = client.count(
    collection_name="appunti_corso",
    exact=True
)

print("Numero di vettori nella collection:", count.count)

Numero di vettori nella collection: 428


A questo punto andiamo a creare un oggetto di ricerca

In [51]:
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3} #prendi i tre vettori più simili
)

#simile alla query
#SELECT TOP 3 *
#FROM appunti_corso
#ORDER BY similarity DESC;

In [52]:
query = "Come funziona una JOIN in SQL?"
results = retriever.invoke(query)

Visualizziamo il risultato

In [53]:
for r in results:
    print(f"\n--- RISULTATO ---")
    print(r.page_content[:800])
    print("\ncorso:", r.metadata.get("corso"))


--- RISULTATO ---
Altre tipologie di Join
Le due query seguenti sono equivalenti:
SELECT dbo.Fatture.*, dbo.Fornitori.Denominazione
FROM   dbo.Fatture
LEFT JOIN dbo.Fornitori
ON dbo.Fatture.IdFornitore = dbo.Fornitori.IdFornitore;
SELECT dbo.Fatture.*, dbo.Fornitori.Denominazione
FROM   dbo.Fornitori
RIGHT JOIN dbo.Fatture
ON dbo.Fornitori.IdFornitore = dbo.Fatture.IdFornitore;

corso: sql

--- RISULTATO ---
Commento
Ricorda: le colonne che utilizzo nella condizioni di JOIN devono contenere
sempre la stessa tipologia di informazione. Ma da solo questo non basta! Ecco
un altro esempio potenzialmente errato.
Mi risulta molto difficile trovare un esercizio per cui questa operazioni abbia
senso
SELECT * 
FROM dbo.Clienti AS C
INNER JOIN dbo.Fornitori AS F
ON C.RegioneResidenza = F.RegioneResidenza;
Consiglio: controlliamo sempre come cambia il conteggio dei dati dopo la JOIN e 
cerchiamo di capire perché. Potrebbero essere svariati motivi

corso: sql

--- RISULTATO ---
Utilizzo di AS nell

Aggiungiamo un formattatore

In [54]:
def format_docs(docs):
    texts = []

    for d in docs:
        corso = d.metadata.get("corso")
        pagina = d.metadata.get("page", "?")
        contenuto = d.page_content

        riga = "[fonte=" + str(corso) + " | pagina=" + str(pagina) + "] \n" + contenuto
        texts.append(riga)

    return "\n\n".join(texts)

In [55]:
print(format_docs(results))

[fonte=sql | pagina=87] 
Altre tipologie di Join
Le due query seguenti sono equivalenti:
SELECT dbo.Fatture.*, dbo.Fornitori.Denominazione
FROM   dbo.Fatture
LEFT JOIN dbo.Fornitori
ON dbo.Fatture.IdFornitore = dbo.Fornitori.IdFornitore;
SELECT dbo.Fatture.*, dbo.Fornitori.Denominazione
FROM   dbo.Fornitori
RIGHT JOIN dbo.Fatture
ON dbo.Fornitori.IdFornitore = dbo.Fatture.IdFornitore;

[fonte=sql | pagina=100] 
Commento
Ricorda: le colonne che utilizzo nella condizioni di JOIN devono contenere
sempre la stessa tipologia di informazione. Ma da solo questo non basta! Ecco
un altro esempio potenzialmente errato.
Mi risulta molto difficile trovare un esercizio per cui questa operazioni abbia
senso
SELECT * 
FROM dbo.Clienti AS C
INNER JOIN dbo.Fornitori AS F
ON C.RegioneResidenza = F.RegioneResidenza;
Consiglio: controlliamo sempre come cambia il conteggio dei dati dopo la JOIN e 
cerchiamo di capire perché. Potrebbero essere svariati motivi

[fonte=sql | pagina=80] 
Utilizzo di AS nella F

O analogamente, con la catena di langchain

In [56]:
context_chain = retriever | format_docs

query = "Come funziona una JOIN in SQL?"
print(context_chain.invoke(query))

[fonte=sql | pagina=87] 
Altre tipologie di Join
Le due query seguenti sono equivalenti:
SELECT dbo.Fatture.*, dbo.Fornitori.Denominazione
FROM   dbo.Fatture
LEFT JOIN dbo.Fornitori
ON dbo.Fatture.IdFornitore = dbo.Fornitori.IdFornitore;
SELECT dbo.Fatture.*, dbo.Fornitori.Denominazione
FROM   dbo.Fornitori
RIGHT JOIN dbo.Fatture
ON dbo.Fornitori.IdFornitore = dbo.Fatture.IdFornitore;

[fonte=sql | pagina=100] 
Commento
Ricorda: le colonne che utilizzo nella condizioni di JOIN devono contenere
sempre la stessa tipologia di informazione. Ma da solo questo non basta! Ecco
un altro esempio potenzialmente errato.
Mi risulta molto difficile trovare un esercizio per cui questa operazioni abbia
senso
SELECT * 
FROM dbo.Clienti AS C
INNER JOIN dbo.Fornitori AS F
ON C.RegioneResidenza = F.RegioneResidenza;
Consiglio: controlliamo sempre come cambia il conteggio dei dati dopo la JOIN e 
cerchiamo di capire perché. Potrebbero essere svariati motivi

[fonte=sql | pagina=80] 
Utilizzo di AS nella F

Integriamo l'uso dell'LLM

In [59]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import init_chat_model
from langchain_core.runnables import RunnableLambda

# 1) retriever dal tuo vectorstore
vectorstore = QdrantVectorStore(
    client=client,
    collection_name="appunti_corso",
    embedding=embeddings
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 2) funzione che formatta
def format_docs(docs):
    texts = []

    for d in docs:
        corso = d.metadata.get("corso")
        pagina = d.metadata.get("page", "?")
        contenuto = d.page_content

        riga = "[fonte=" + str(corso) + " | pagina=" + str(pagina) + "] " + contenuto
        texts.append(riga)

    return "\n\n".join(texts)

# 3) prompt RAG
prompt = ChatPromptTemplate.from_messages([
    ("system", "Rispondi in italiano usando SOLO il contesto. Se non trovi la risposta nel contesto, di' 'Non lo so'."),
    ("human", "Domanda: {question}\n\nContesto:\n{context}")
])

# 4) LLM (serve OPENAI_API_KEY nell'ambiente)
llm = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    temperature = 0,
    api_key = my_api_key,
    max_tokens=1000,
)

# 5) chain: question -> retriever -> context + question -> prompt -> llm -> testo
rag_chain = (
    {
        "context": RunnableLambda(lambda x: x["question"]) | retriever | format_docs,
        "question": RunnableLambda(lambda x: x["question"])
    }
    | prompt
    | llm
    | StrOutputParser()
)

# 6) test
print(rag_chain.invoke(input ={"question":"Come funziona una JOIN in SQL?"}))

Una JOIN in SQL permette di combinare righe di due o più tabelle in base a una condizione specificata. Ad esempio, una LEFT JOIN restituisce tutte le righe dalla tabella a sinistra e le righe corrispondenti dalla tabella a destra, mentre una RIGHT JOIN fa l'opposto. È importante che le colonne utilizzate nelle condizioni di JOIN contengano la stessa tipologia di informazione. Inoltre, è consigliabile controllare come cambia il conteggio dei dati dopo la JOIN per comprendere meglio il risultato.


Sintassi più leggera

In [60]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import init_chat_model
from langchain_core.runnables import RunnableLambda

# 1) retriever dal tuo vectorstore
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 2) funzione che formatta
def format_docs(docs):
    texts = []

    for d in docs:
        corso = d.metadata.get("corso")
        pagina = d.metadata.get("page", "?")
        contenuto = d.page_content

        riga = "[fonte=" + str(corso) + " | pagina=" + str(pagina) + "] " + contenuto
        texts.append(riga)

    return "\n\n".join(texts)

# 3) prompt RAG
prompt = ChatPromptTemplate.from_messages([
    ("system", "Rispondi in italiano usando SOLO il contesto. Se non trovi la risposta nel contesto, di' 'Non lo so'."),
    ("human", "Domanda: {question}\n\nContesto:\n{context}")
])

# 4) LLM (serve OPENAI_API_KEY nell'ambiente)
llm = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    api_key = my_api_key,
    temperature = 0,
    max_tokens=1000,
)

# 5) chain: question -> retriever -> context + question -> prompt -> llm -> testo
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke("Come funziona una JOIN in SQL?"))


Una JOIN in SQL permette di combinare righe di due o più tabelle in base a una condizione specificata. Ad esempio, una LEFT JOIN restituisce tutte le righe dalla tabella a sinistra e le righe corrispondenti dalla tabella a destra, mentre una RIGHT JOIN fa l'opposto. È importante che le colonne utilizzate nelle condizioni di JOIN contengano la stessa tipologia di informazione. Inoltre, è consigliabile controllare come cambia il conteggio dei dati dopo la JOIN per comprendere meglio il risultato.


# Database

## Parte relativa al DB Vettoriale

In [ ]:
from qdrant_client import QdrantClient
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_qdrant import QdrantVectorStore

qdrant_client = QdrantClient(path=r"C:\Users\ianto\Desktop\Langchain\qdrant_db")

embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-small-en-v1.5")

vectorstore = QdrantVectorStore(
    client=qdrant_client,
    collection_name="appunti_corso",
    embedding=embeddings,
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

def format_docs(docs):
    texts = []
    for d in docs:
        corso = d.metadata.get("corso")
        pagina = d.metadata.get("page", "?")
        contenuto = d.page_content
        texts.append(f"[fonte={corso} | pagina={pagina}] {contenuto}")
    return "\n\n".join(texts)

def search_pdf(question: str) -> str:
    docs = retriever.invoke(question)
    if not docs:
        return "NESSUN RISULTATO"
    return format_docs(docs)

## Database Relazionale

In [ ]:
#Su SQL Sever
#CREATE LOGIN solo_lettura
#WITH PASSWORD = 'LaTuaPassword';
#GO
#USE AdventureWorks2019
#GO
#CREATE USER solo_lettura
#FOR LOGIN solo_lettura;
#GO
#ALTER ROLE db_datareader ADD MEMBER solo_lettura;

In [ ]:
server_name = r'LAPTOP-UDP6N0UL\SQLEXPRESS'
database_name = 'AdventureWorks2019'
utente = getpass.getpass(prompt='Inserisci nome utente: ')
password = getpass.getpass(prompt='Inserisci la password: ')

In [ ]:
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Engine
import re

#ricordarsi di aggiungere in produzione &Encrypt=yes&TrustServerCertificate=no
conn_str = f'mssql+pyodbc://{utente}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(conn_str)

#Sicurezza aggiuntiva, ma comunque fondamentale entrare con utente in solo lettura. Ricordiamo che su SQL Server possiamo lanciare procedure senza exec
#ma indicando soltanto il nome
FORBIDDEN = re.compile(r"\b(insert|update|delete|merge|drop|alter|create|truncate|exec|execute)\b", re.I)

def sql_server_query(query, params = {}, mode = "r"):
    if mode not in {"r", "m"}:
        raise ValueError(f"Valore di mode non valido. Attesi 'r' o 'm'.")

    q = query.strip()

    if FORBIDDEN.search(q):
        return "BLOCCATO: sono consentite solo query SELECT (read-only)."

    if not re.match(r"^\s*select\b", q, flags=re.I):
        return "BLOCCATO: devi usare una query SELECT."

    if mode == "r":
        if not re.search(r"^\s*select\s+top\s+\d+", q, flags=re.I):
            q = re.sub(r"^\s*select\b", "SELECT TOP 50", q, flags=re.I)

    try:
        with engine.connect() as conn:
            res = conn.execute(text(q),params)
            if mode == "r":
                rows = res.fetchmany(50)
            if mode == "m":
                rows = res.fetchall()
            cols = list(res.keys())
    except Exception as e:
        return f"ERRORE SQL: {e}"

    if not rows:
        return "RISULTATO: 0 righe."

    # Formattatore di testo
    lines = []
    lines.append(" | ".join(cols))
    lines.append("-"*50)
    for r in rows:
        lines.append(" | ".join("" if v is None else str(v) for v in r))
    return "\n\n".join(lines)

In [ ]:
print(sql_server_query("SELECT * FROM SALES.CUSTOMER"))

definiamo due funzioni che diventeranno dei tool

In [ ]:
#il filtro  SCHEMA <> 'dbo' è specifico per il DB AdventureWorks2019

def sql_list_tables(_):
    q = """
    SELECT
      TABLE_SCHEMA, TABLE_NAME
    FROM INFORMATION_SCHEMA.TABLES
    WHERE TABLE_SCHEMA <> 'dbo';
    """
    return sql_server_query(q, mode = "m", params = {})

def sql_table_schema(table_fullname):
    name = table_fullname.strip()
    if "." not in name:
        return "FORMATO: usa 'schema.tabella' (es. dbo.Clienti)."

    schema, table = name.split(".", 1)
    q = """
    SELECT
      COLUMN_NAME, DATA_TYPE, IS_NULLABLE, CHARACTER_MAXIMUM_LENGTH
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_SCHEMA = :schema
      AND TABLE_NAME   = :table
    ORDER BY ORDINAL_POSITION;
    """
    return sql_server_query(q, mode = "m", params={"schema": schema, "table": table})

Proviamo le due funzioni

In [ ]:
print(sql_list_tables(_))

In [ ]:
print(sql_table_schema("production.product"))

## Agente

In [ ]:
from typing import List, Optional

from langchain_core.tools import Tool
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model


llm = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai",
    temperature=0,
    max_tokens=1000,
)


tools = [
    Tool(
        name="search_pdf",
        func=search_pdf,
        description="Cerca negli appunti (PDF indicizzati) e restituisce estratti con fonte/pagina."
    ),
    Tool(
        name="sql_list_tables",
        func=sql_list_tables,
        description="Elenca tabelle e viste del database SQL Server locale."
    ),
    Tool(
        name="sql_table_schema",
        func=sql_table_schema,
        description="Mostra le colonne di una tabella. Input: 'schema.tabella' (es. 'dbo.Clienti')."
    ),
    Tool(
        name="sql_server_query",
        func=sql_server_query,
        description="Esegue una query SELECT su SQL Server locale (read-only). L'agente deve limitare i risultati."
    ),
]


system_prompt = """
    Sei un agente didattico per SQL/Python.
    Strumenti:
    - search_pdf: per concetti del corso
    - sql_list_tables/sql_table_schema/sql_server_query: per interrogare il DB SQL Server locale

    Regole:
    1) Se la domanda riguarda SQL/Python come spiegati nel corso, usa 'search_pdf' e fermati.
    2) Se la domanda chiede dati reali, usa gli strumenti SQL.
    3) Usa SOLO query SELECT e limita sempre l'output (TOP 50 è OK).
    4) Se ti manca lo schema, chiama prima sql_list_tables o sql_table_schema.
    5) Se una cosa non è nei PDF e non è nel DB, dillo chiaramente.
    6) Dopo aver specificato che non hai informazioni, aggiungi comunque la rispsota.
    """

agent_executor = create_agent(llm, tools)

question = "Quanti clienti abbiamo?"

result = agent_executor.invoke({
    "messages": [
        ("system", system_prompt),
        ("user", question),
    ]
})

print(result["messages"][-1].content)

In [ ]:
for m in result["messages"]:
    print(type(m), getattr(m, "content", ""), getattr(m, "tool_calls", None))

In [ ]:
engine.dispose()

# SQLDatabase di LangChain

In [ ]:
import os
import getpass
secret_key = getpass.getpass(prompt='Inserisci la chiave: ')

In [ ]:
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from sqlalchemy import create_engine, inspect
from langchain.chat_models import init_chat_model
import urllib

server_name = r'LAPTOP-UDP6N0UL\SQLEXPRESS'
database_name = 'AdventureWorks2019'
utente = "solo_lettura"
password = 'LaTuaPassword'

# ===== CONFIGURAZIONE =====
#ricordarsi di aggiungere in produzione &Encrypt=yes&TrustServerCertificate=no
conn_str = f'mssql+pyodbc://{utente}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(conn_str)

# ===== LLM =====
llm = ChatAnthropic(
    api_key=secret_key,
    model="claude-opus-4-5-20251101"
)


In [ ]:
# Crea oggetto database di LangChain CON TUTTE LE TABELLE
db = SQLDatabase(engine, schema="sales")

# ===== AGENTE =====
agent = create_sql_agent(llm=llm, db=db)

# ===== USA =====

In [ ]:
# 4. Domanda in linguaggio naturale
response = agent.invoke({"input": "Qual è il prezzo medio del prodotto 776? Rispondi solo con un numero"})
print(response["output"])
